# JSON Validation Java Tutorial
In this tutorial, we will learn how to validate a JSON instance and create a customized Format Validator using the Java classes in the com.ndi package. First we will start will a brief introduction to JSON Schema and the format tag.

## A brief introduction to JSON Schema

For the NDI project, JSON file is used to represent an instance of our ndi classes (or our ndi object), such as ndi_session, ndi_daqreader, and so on. JSON files consists of a series of key-value pairs, each of which represents the class's field and its value. Therefore, before storing a JSON file into a database, it is important to ensure that the JSON instance is valid. The java packages com.ndi within the ndi-validator-java.jar file provide document validation functions that we can call from MATLAB to validate a JSON instance (this is precisely what *ndi_validate.m* does")

A JSON Schmea is special JSON file that is used to specify the expected data type for each key of a JSON Document. This might appear very abstract right now, so we will provide a concrete example:

Say that we have an instance of ndi_subject which consits of two fields: **local_identifier** and **description**, we can represents this instance of ndi_subject with the following JSON file: *sample-ndi-subject.json*

Clearly both local_identifier and description has to be string, something like such will be invalid

In order to enforce that both the field **local_identifier** and the field **description** needs to be string, we can create a JSON schema document, let's call it *ndi_document_subject_schema.json*

The **"$schema"** tag tells us the json_schema specification we are using. You can read the specification document using this link. The  **"id"** tag tells us the corresponding json document it is trying to enfor rule upon. The **"title"** tag specifies the name of the object. All of the three above tags are semantic tags. That is, they does not impact the way our JSON Validator validates our JSON document.

The **"type"** tag specifies the expected data type of the data this JSON file represents. Here *ndi_document_subject_schema.json* represents a MATLAB object, so we let the type to be "object". Next within the properties tag, we need to specify the expected data type of each fields of this object. Here we want both the fields "local_identifier" and the field "description" to be a string

You can read more about the key word an the expected structure of the JSON Schema document through this link *https://json-schema.org/understanding-json-schema/*

## Validating JSON Document

Our next task is the use the Validation class in the com.ndi package to validate the JSON Document. First we need to import Validator class from the com.ndi package. 

In [1]:
import com.ndi.Validator;

There are a couple ways to construct a new instance of the Validator class. Here are the method signature of the Validaotr's constructor

The second constructor is a bit complicated, we will get into that later in the tutorial. 

**The first constructor takes three parameters**:

**document**: this can either be the path to the .json file we want to validate, or the content of the JSON file

**schema**: this can either be the path to the .json file that is the corresponding json schema of the JSON document file we passed in as our first parameter.

**schema**: if both the document parameter and the schema parameter are both the path to the .json file, this should be false. Otherwise, if both of the two parameters represent the actual JSON file content, this parameter should be true

Again an example will make everything much cleaer. Let's try to validate the *sample-ndi-subject.json* we have created above against the *ndi_document_subject_schema.json*

In [2]:
//the file content of the sample-ndi-subject.json

String document = "{\n" +
                "    local_identifier : \"subject@brandeis.edu\",\n" +
                "    description : \"this is a dummy subject\"\n" +
                "}";

In [3]:
//the file content of the ndi_document_subject_schema.json

String schema = "{\n" +
            "    \"$schema\": \"http://json-schema.org/draft/2019-09/schema#\",\n" +
            "    \"id\": \"$NDISCHEMAPATH\\/ndi_document_subject_schema.json\",\n" +
            "    \"title\": \"ndi_document_subject\",\n" +
            "    \"type\": \"object\",\n" +
            "    \"properties\": {\n" +
            "        \"local_identifier\": {\n" +
            "            \"type\": \"string\"\n" +
            "        },\n" +
            "        \"description\": {\n" +
            "            \"type\": \"string\"\n" +
            "        }\n" +
            "    }\n" +
            "}";

Since they represents the actual JSON file content, isJSONContent should be true. We just simply need to pass those into our Validator's constructor:

In [4]:
Validator ndi_subject = new Validator(document, schema, true);

null


Next we can use the **getReport() methods: public HashMap<String, String> getReport()** to get a detailed report of our validation. As you can see, it will return an instance of java.util.HashMap. 

In [5]:
ndi_subject.getReport()

{}

We get an empty HashMap because our JSON does not contain any type error. Both the local_identifier fields and the description field have values that are string. Therefore, it is valid. Next, let's see what happen if we entered invalid value into those fields

In [6]:
//the file content of the sample-ndi-subject-wrong-type.json

String document = "{\n" +
                "    local_identifier : 153,\n" +
                "    description : \"this is a dummy subject\"\n" +
                "}";

In [7]:
Validator ndi_subject = new Validator(document, schema, true);
ndi_subject.getReport();

null


{#/local_identifier=#/local_identifier: expected type: String, found: Integer}

The report returns a key-value pairs, which tells us for the field "local_identifier", we have entered an Integer, but in our schema specification, we require it to be a String. Therefore, the validation fails just as what we would have expected. We can valdate any JSON object with a given JSON schema using the Validator class. This concludes the first part of the tutorial